Script to compute p-values on differences in alpha power in specific regions for all brain types

Previous scripts (to compute alpha power across 100simulations):
    HF_100simluations with PSD
    LF_100simluations with PSD
    depr_100simluations with PSD
    health_100simluations with PSD

In [1]:
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


In [2]:
from tvb.simulator.lab import *
import h5py
import os
import matplotlib.pyplot as plt

import scipy 
import scipy.fftpack
import seaborn as sns

import scipy 
import scipy.fftpack
from scipy.fft import fft, ifft
from scipy import signal
from scipy.integrate import simps
from fooof import FOOOF
from scipy import stats

import pandas as pd
from statistics import stdev

In [3]:
os.chdir("C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Wilson Cowan/DATA")

In [4]:
#import connectivity file to know which region corresponds to which index in the matrices
filename = "Connectivity.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    regions_list = list(f['region_labels'])

regions = np.array(regions_list)

Keys: <KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>


In [5]:
print(regions)

['rA1' 'rA2' 'rAMYG' 'rCCA' 'rCCP' 'rCCR' 'rCCS' 'rFEF' 'rG' 'rHC' 'rIA'
 'rIP' 'rM1' 'rPCI' 'rPCIP' 'rPCM' 'rPCS' 'rPFCCL' 'rPFCDL' 'rPFCDM'
 'rPFCM' 'rPFCORB' 'rPFCPOL' 'rPFCVL' 'rPHC' 'rPMCDL' 'rPMCM' 'rPMCVL'
 'rS1' 'rS2' 'rTCC' 'rTCI' 'rTCPOL' 'rTCS' 'rTCV' 'rV1' 'rV2' 'rCC' 'lA1'
 'lA2' 'lAMYG' 'lCCA' 'lCCP' 'lCCR' 'lCCS' 'lFEF' 'lG' 'lHC' 'lIA' 'lIP'
 'lM1' 'lPCI' 'lPCIP' 'lPCM' 'lPCS' 'lPFCCL' 'lPFCDL' 'lPFCDM' 'lPFCM'
 'lPFCORB' 'lPFCPOL' 'lPFCVL' 'lPHC' 'lPMCDL' 'lPMCM' 'lPMCVL' 'lS1' 'lS2'
 'lTCC' 'lTCI' 'lTCPOL' 'lTCS' 'lTCV' 'lV1' 'lV2' 'lCC']


In [5]:
#import alpha power
mydir = "C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets/All_results"
    
filename = mydir + "/Health_Time.h5"
with h5py.File(filename, "r") as f:
    healthtime = f["time"]
    healthtime = np.array(healthtime)
    
filename = mydir + "/Health_power_alpha_overall.h5"
with h5py.File(filename, "r") as f:
    alpha_all = f["data"]
    alpha_all = np.array(alpha_all)

filename = mydir + "/Health_power_alpha_lDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_left = f["data"]
    alpha_left = np.array(alpha_left)
    
filename = mydir + "/Health_power_alpha_rDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_right = f["data"]
    alpha_right = np.array(alpha_right)
    
filename = mydir + "/Depressed_power_alpha_overall.h5"
with h5py.File(filename, "r") as f:
    alpha_all_dep = f["data"]
    dep_all = np.array(alpha_all_dep)
    

filename = mydir + "/Depressed_power_alpha_lDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_left_depr = f["data"]
    alpha_left_depr = np.array(alpha_left_depr)
    
filename = mydir + "/Depressed_power_alpha_rDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_right_depr = f["data"]
    alpha_right_depr = np.array(alpha_right_depr)
    
    

filename = mydir + "/HF_power_alpha_overall.h5"
with h5py.File(filename, "r") as f:
    alpha_all_HF = f["data"]
    HF_all = np.array(alpha_all_HF)
    

filename = mydir + "/HF_power_alpha_lDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_left_HF = f["data"]
    alpha_left_HF = np.array(alpha_left_HF)
    
filename = mydir + "/HF_power_alpha_rDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_right_HF = f["data"]
    alpha_right_HF = np.array(alpha_right_HF)

filename = mydir + "/LF_power_alpha_overall.h5"
with h5py.File(filename, "r") as f:
    alpha_all_LF = f["data"]
    LF_all = np.array(alpha_all_LF)
    

filename = mydir + "/LF_power_alpha_lDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_left_LF = f["data"]
    alpha_left_LF = np.array(alpha_left_HF)
    
filename = mydir + "/LF_power_alpha_rDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_right_LF = f["data"]
    alpha_right_LF = np.array(alpha_right_HF)
    

filename = mydir + "/Health_Frequencies.h5"
with h5py.File(filename, "r") as f:
    freqs = f["data"]
    freqs = np.array(freqs)

In [6]:
print(freqs)

[ 8.203125  8.59375   8.984375  9.375     9.765625 10.15625  10.546875
 10.9375   11.328125 11.71875  12.109375]


        #store PSDs
        #psd for all brain 
        for x in range(5):            
            low, high = 8+x, 9+x
            if x == 5:
                low, high = 8, 12

eerste rij van output file = 8-9Hz
tweede rij = 9-10
derde = 10-11
vierde = 11-12
laatste = 8-12

In [7]:
import statistics
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

statsx = importr('stats')

def EFsize(c0, c1):
    d = (mean(c0) - mean(c1)) / (sqrt((statistics.stdev(c0) ** 2 + statistics.stdev(c1) ** 2) / 2))
    return d


In [8]:
mydir = "C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets/All_results/Alpha_power"

In [14]:
#left frontal alfa 

for i in range(5):
    #first: compare all lowest frequencies 
    #4 means, 4 sd's, 6p-values, 6EFs, 6 corrected p-values
    
    health = alpha_left[:,i]
    depr = alpha_left_depr[:,i]
    LF = alpha_left_LF[:,i]
    HF = alpha_left_HF[:,i]

    array = numpy.zeros(shape = (1, 26))

    array[0][0] = mean(health)
    array[0][1] = mean(depr)
    array[0][2] = mean(LF)
    array[0][3] = mean(HF)
    array[0][4] = np.std(health)
    array[0][5] = np.std(depr)
    array[0][6] = np.std(LF)
    array[0][7] = np.std(HF)
    array[0][8] = stats.ttest_ind(health, depr).pvalue
    array[0][9] = stats.ttest_ind(health, LF).pvalue
    array[0][10] = stats.ttest_ind(health, HF).pvalue
    array[0][11] = stats.ttest_ind(LF, depr).pvalue
    array[0][12] = stats.ttest_ind(HF, depr).pvalue
    array[0][13] = stats.ttest_ind(HF, LF).pvalue
    array[0][14] = EFsize(health, depr)
    array[0][15] = EFsize(health, LF)
    array[0][16] = EFsize(health, HF)
    array[0][17] = EFsize(LF, depr)
    array[0][18] = EFsize(HF, depr)
    array[0][19] = EFsize(HF, LF)
    array[0][20] = stats.ttest_ind(health, depr).statistic
    array[0][21] = stats.ttest_ind(health, LF).statistic
    array[0][22] = stats.ttest_ind(health, HF).statistic
    array[0][23] = stats.ttest_ind(LF, depr).statistic
    array[0][24] = stats.ttest_ind(HF, depr).statistic
    array[0][25] = stats.ttest_ind(HF, LF).statistic

    #make new array with adjusted p-values!
    if i == 0:
        array0 = array
    elif i ==1:
        array1 = array
    elif i == 2:
        array2 = array
    elif i == 3:
        array3 = array
    else:
        array4 = array

tuple = (array0, array1, array2, array3, array4)
total = np.vstack(tuple)



parray = total[:, [8,9,10,11,12,13]]

padj = np.copy(parray)

padj[:,0] = statsx.p_adjust(FloatVector(parray[:,0]), method = 'holm')
padj[:,1] = statsx.p_adjust(FloatVector(parray[:,1]), method = 'holm')
padj[:,2] = statsx.p_adjust(FloatVector(parray[:,2]), method = 'holm')
padj[:,3] = statsx.p_adjust(FloatVector(parray[:,3]), method = 'holm')
padj[:,4] = statsx.p_adjust(FloatVector(parray[:,4]), method = 'holm')
padj[:,5] = statsx.p_adjust(FloatVector(parray[:,5]), method = 'holm')

tuplex = (total, padj)
total_adj = np.column_stack(tuplex)

file_name = mydir + "/left frontal alfa p-values_with T.csv"
outcome = pd.DataFrame.from_records(total_adj)
outcome.columns = ["mean health","mean depr","meanLF", "meanHF", "std health","std depr","std LF","std HF","p H_D","p H_LF",
                  "p H_HF","p LF_depr","p HF_depr","p HF_LF","ES H_HF", "ES H_LF","ES H_HF","ES LF_depr","ES HF_depr","ES HF_LF",
                  "t adj H_depr", "t adj H_LF", "t adj H_HF","t adj LF_depr", "t adj HF_depr", "t adj HF_LF",
                   "p adj H_depr", "p adj H_LF", "p adj H_HF","p adj LF_depr", "p adj HF_depr", "p adj HF_LF"]
outcome.to_csv(path_or_buf = file_name, index = False)

In [15]:
#right frontal alfa 

for i in range(5):
    #first: compare all lowest frequencies 
    #4 means, 4 sd's, 6p-values, 6EFs, 6 corrected p-values
    
    health = alpha_right[:,i]
    depr = alpha_right_depr[:,i]
    LF = alpha_right_LF[:,i]
    HF = alpha_right_HF[:,i]

    array = numpy.zeros(shape = (1, 26))

    array[0][0] = mean(health)
    array[0][1] = mean(depr)
    array[0][2] = mean(LF)
    array[0][3] = mean(HF)
    array[0][4] = np.std(health)
    array[0][5] = np.std(depr)
    array[0][6] = np.std(LF)
    array[0][7] = np.std(HF)
    array[0][8] = stats.ttest_ind(health, depr).pvalue
    array[0][9] = stats.ttest_ind(health, LF).pvalue
    array[0][10] = stats.ttest_ind(health, HF).pvalue
    array[0][11] = stats.ttest_ind(LF, depr).pvalue
    array[0][12] = stats.ttest_ind(HF, depr).pvalue
    array[0][13] = stats.ttest_ind(HF, LF).pvalue
    array[0][14] = EFsize(health, depr)
    array[0][15] = EFsize(health, LF)
    array[0][16] = EFsize(health, HF)
    array[0][17] = EFsize(LF, depr)
    array[0][18] = EFsize(HF, depr)
    array[0][19] = EFsize(HF, LF)
    array[0][20] = stats.ttest_ind(health, depr).statistic
    array[0][21] = stats.ttest_ind(health, LF).statistic
    array[0][22] = stats.ttest_ind(health, HF).statistic
    array[0][23] = stats.ttest_ind(LF, depr).statistic
    array[0][24] = stats.ttest_ind(HF, depr).statistic
    array[0][25] = stats.ttest_ind(HF, LF).statistic

    #make new array with adjusted p-values!
    if i == 0:
        array0 = array
    elif i ==1:
        array1 = array
    elif i == 2:
        array2 = array
    elif i == 3:
        array3 = array
    else:
        array4 = array

tuple = (array0, array1, array2, array3, array4)
total = np.vstack(tuple)

parray = total[:, [8,9,10,11,12,13]]

padj = np.copy(parray)

padj[:,0] = statsx.p_adjust(FloatVector(parray[:,0]), method = 'holm')
padj[:,1] = statsx.p_adjust(FloatVector(parray[:,1]), method = 'holm')
padj[:,2] = statsx.p_adjust(FloatVector(parray[:,2]), method = 'holm')
padj[:,3] = statsx.p_adjust(FloatVector(parray[:,3]), method = 'holm')
padj[:,4] = statsx.p_adjust(FloatVector(parray[:,4]), method = 'holm')
padj[:,5] = statsx.p_adjust(FloatVector(parray[:,5]), method = 'holm')

tuplex = (total, padj)
total_adj = np.column_stack(tuplex)

file_name = mydir + "/right frontal alfa p-values_with T.csv"
outcome = pd.DataFrame.from_records(total_adj)
outcome.columns = ["mean health","mean depr","meanLF", "meanHF", "std health","std depr","std LF","std HF","p H_D","p H_LF",
                  "p H_HF","p LF_depr","p HF_depr","p HF_LF","ES H_HF", "ES H_LF","ES H_HF","ES LF_depr","ES HF_depr","ES HF_LF",
                  "t adj H_depr", "t adj H_LF", "t adj H_HF","t adj LF_depr", "t adj HF_depr", "t adj HF_LF",
                   "p adj H_depr", "p adj H_LF", "p adj H_HF","p adj LF_depr", "p adj HF_depr", "p adj HF_LF"]
outcome.to_csv(path_or_buf = file_name, index = False)


In [16]:
#left right asymmetry 
#right frontal alfa 

as_depr = alpha_left_depr / alpha_right_depr
as_health = alpha_left / alpha_right
as_LF = alpha_left_LF / alpha_right_LF
as_HF = alpha_left_HF / alpha_right_HF

for i in range(5):
    #first: compare all lowest frequencies 
    #4 means, 4 sd's, 6p-values, 6EFs, 6 corrected p-values
    
    health = as_health[:,i]
    depr = as_depr[:,i]
    LF = as_LF[:,i]
    HF = as_HF[:,i]

    array = numpy.zeros(shape = (1, 26))

    array[0][0] = mean(health)
    array[0][1] = mean(depr)
    array[0][2] = mean(LF)
    array[0][3] = mean(HF)
    array[0][4] = np.std(health)
    array[0][5] = np.std(depr)
    array[0][6] = np.std(LF)
    array[0][7] = np.std(HF)
    array[0][8] = stats.ttest_ind(health, depr).pvalue
    array[0][9] = stats.ttest_ind(health, LF).pvalue
    array[0][10] = stats.ttest_ind(health, HF).pvalue
    array[0][11] = stats.ttest_ind(LF, depr).pvalue
    array[0][12] = stats.ttest_ind(HF, depr).pvalue
    array[0][13] = stats.ttest_ind(HF, LF).pvalue
    array[0][14] = EFsize(health, depr)
    array[0][15] = EFsize(health, LF)
    array[0][16] = EFsize(health, HF)
    array[0][17] = EFsize(LF, depr)
    array[0][18] = EFsize(HF, depr)
    array[0][19] = EFsize(HF, LF)
    array[0][20] = stats.ttest_ind(health, depr).statistic
    array[0][21] = stats.ttest_ind(health, LF).statistic
    array[0][22] = stats.ttest_ind(health, HF).statistic
    array[0][23] = stats.ttest_ind(LF, depr).statistic
    array[0][24] = stats.ttest_ind(HF, depr).statistic
    array[0][25] = stats.ttest_ind(HF, LF).statistic

    #make new array with adjusted p-values!
    if i == 0:
        array0 = array
    elif i ==1:
        array1 = array
    elif i == 2:
        array2 = array
    elif i == 3:
        array3 = array
    else:
        array4 = array

tuple = (array0, array1, array2, array3, array4)
total = np.vstack(tuple)

parray = total[:, [8,9,10,11,12,13]]

padj = np.copy(parray)

padj[:,0] = statsx.p_adjust(FloatVector(parray[:,0]), method = 'holm')
padj[:,1] = statsx.p_adjust(FloatVector(parray[:,1]), method = 'holm')
padj[:,2] = statsx.p_adjust(FloatVector(parray[:,2]), method = 'holm')
padj[:,3] = statsx.p_adjust(FloatVector(parray[:,3]), method = 'holm')
padj[:,4] = statsx.p_adjust(FloatVector(parray[:,4]), method = 'holm')
padj[:,5] = statsx.p_adjust(FloatVector(parray[:,5]), method = 'holm')

tuplex = (total, padj)
total_adj = np.column_stack(tuplex)

file_name = mydir + "/L_F_asymmetry frontal alfa p-values_with T.csv"
outcome = pd.DataFrame.from_records(total_adj)
outcome.columns = ["mean health","mean depr","meanLF", "meanHF", "std health","std depr","std LF","std HF","p H_D","p H_LF",
                  "p H_HF","p LF_depr","p HF_depr","p HF_LF","ES H_HF", "ES H_LF","ES H_HF","ES LF_depr","ES HF_depr","ES HF_LF",
                  "t adj H_depr", "t adj H_LF", "t adj H_HF","t adj LF_depr", "t adj HF_depr", "t adj HF_LF",
                   "p adj H_depr", "p adj H_LF", "p adj H_HF","p adj LF_depr", "p adj HF_depr", "p adj HF_LF"]
outcome.to_csv(path_or_buf = file_name, index = False)
